In [7]:
from src.data.data_loader import BrisT1DDataLoader
from src.tuning.benchmark import impute_missing_values

# Use 02, 03, 04
patient = "p02"

# Split up data into day and night
loader = BrisT1DDataLoader(use_cached=True)

train_df = loader.train_data[loader.train_data["p_num"] == patient]
test_df = loader.validation_data[loader.validation_data["p_num"] == patient]

TIME_STEP_SIZE = (
    train_df["datetime"].iloc[1] - train_df["datetime"].iloc[0]
).components.minutes


if TIME_STEP_SIZE != 5 and TIME_STEP_SIZE != 15:
    error = """
    First time step is not 5 or 15 minutes. Look at the most common time step size.
    """


def reduce_features(df):
    # Make sure index is set to datetime
    p_df = df.iloc[:]
    # p_df.set_index("datetime", inplace=True)
    # p_df = p_df.asfreq('5T', method='pad')

    # Reduce features
    y_feature = ["bg-0:00"]
    x_features = [
        # "hr-0:00", # -> has NaNs
        "steps-0:00",
        "cals-0:00",
        "carbs-0:00",
        "cob",
        "carb_availability",
        "insulin_availability",
        "iob",
    ]
    p_df = p_df[x_features + y_feature]

    # Impute with default methods
    p_df = impute_missing_values(p_df, columns=x_features)
    p_df = impute_missing_values(p_df, columns=y_feature)

    y, X = p_df[y_feature], p_df[x_features]
    return y, X


# train_df
# y_train, X_train = reduce_features(train_df)
# y_train
# train_df["carbs-0:00"].sample(1000)
# X_train["carbs-0:00"].sample(1000)

13012   NaN
12799   NaN
14610   NaN
10373   NaN
11174   NaN
         ..
25051   NaN
12657   NaN
12880   NaN
9465    NaN
22670   NaN
Name: carbs-0:00, Length: 1000, dtype: float64

## TODO
1. Calculate CR and ISF
2. Convert bg from mmol/L (millimoles per liter) to mg/dL (milligrams per deciliter)
3. Find out the trend of DEXCOM: https://www.dexcom.com/all-access/dexcom-cgm-explained/trend-arrow-and-treatment-decisions
4. Process carbs to `ANNOUNCE_MEAL`. (I guess there is no way to capture Portein here, which is used in the model)
    - Filter out everything that is below 5 grams
    - 5-20 g will be considered `INTERVENTION_SNACKS`
    - greater than 20 g will be considered `ANNOUNCE_MEAL`
5. Process does units to `DOSE_INSULIN`

## Calulcate CR and ISF
 - CR: carb/insulin ratio (how many carbs a insulin can cover)
     - Find out window where there is both insulin and food announcement 
 - ISF: blood sugar/insulin ratio (how much glucose drop for each unit of insulin)
    - Need to find out a large dose of insulin with no food announcement
     

Do we need to find out 5 windows for each patient? Like ISF for early morn, breakfast and so on....
Probably would be hard to figure that out?



```
{
    "startOffsetTt": 14400,
    "name": "Early morn",
    "carbRatio": 13.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 100,
},
{
    "startOffsetTt": 23400,
    "name": "Breakfast",
    "carbRatio": 11.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 105,
},
{
    "startOffsetTt": 37800,
    "name": "Lunch",
    "carbRatio": 13.5,
    "isf": 18,
    "insulinResistance": 1,
    "target": 110,
},
{
    "startOffsetTt": 52200,
    "name": "Aft&Dinner",
    "carbRatio": 13.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 110,
},
{
    "startOffsetTt": 79200,
    "name": "Night",
    "carbRatio": 13.5,
    "isf": 85,
    "insulinResistance": 1,
    "target": 110,
}
```